In [19]:
import pandas as pd
import datetime
import sys
from google.protobuf.json_format import MessageToJson
import auto_generated.gtfs_realtime_pb2, auto_generated.nyct_subway_pb2
import json
import pymongo
import requests

In [97]:
#	client = pymongo.MongoClient(
#	       "mongodb://localhost:27017/gtfs?retryWrites=true&w=majority")
#	db = client.gtfs
r = requests.get(f"https://api-endpoint.mta.info/Dataservice/mtagtfsfeeds/nyct%2Fgtfs",
			 headers={"x-api-key": "kDjAqBL0oj2xaVN9F4AdM4fFwgsSYfRY7D6BjYtx"})
gtfs_realtime = auto_generated.gtfs_realtime_pb2.FeedMessage()

gtfs_realtime.ParseFromString(r.content)
message_as_json = json.loads(MessageToJson(gtfs_realtime))

		# add the header info to each event
#		events = message_as_json['entity']
#		for e in events:
#			if 'vehicle' in e:
#				e['header'] = message_as_json['header'].copy()
#				db.events.insert(e)

In [31]:
names = pd.read_csv('../../static data/stops.txt', index_col='stop_id')['stop_name']
names

stop_id
101     Van Cortlandt Park - 242 St
101N    Van Cortlandt Park - 242 St
101S    Van Cortlandt Park - 242 St
103                          238 St
103N                         238 St
                   ...             
S30N                  Tompkinsville
S30S                  Tompkinsville
S31                       St George
S31N                      St George
S31S                      St George
Name: stop_name, Length: 1503, dtype: object

In [33]:
names['115N']

'137 St - City College'

In [98]:
parsed = [{'stop_time': m['arrival']['time'] if 'arrival' in m else m['departure']['time'],
   'stop_id': m['stopId'],
  'origin_time': pd.to_datetime(
      d['tripUpdate']['trip']['startDate'] + ' ' + d['tripUpdate']['trip']['[nyctTripDescriptor]']['trainId'].split()[1].strip(' ').strip('+')
  ) }\
 for d in message_as_json['entity'] if 'tripUpdate' in d\
 and 'SFT/242' in d['tripUpdate']['trip']['[nyctTripDescriptor]']['trainId']\
  and len(d['tripUpdate']['stopTimeUpdate']) > 14\
 for m in d['tripUpdate']['stopTimeUpdate']]
df = pd.DataFrame.from_records(parsed).join(names,on='stop_id').set_index('origin_time')
# from utc to NYC time. lazy should use timezones?
df['stop_time'] = pd.to_datetime(df['stop_time'], unit='s') - pd.Timedelta(hours=4)

In [102]:
df.loc['2020-08-05 12:33:00'].to_csv('run2.csv')

In [52]:
pd.to_datetime('20200805 12:01')

Timestamp('2020-08-05 12:01:00')

In [114]:
run1 = pd.read_csv('run1.csv', parse_dates=['stop_time'])
run2 = pd.read_csv('run2.csv', parse_dates=['stop_time'])
run1['stop_time']

0    2020-08-05 11:34:30
1    2020-08-05 11:37:00
2    2020-08-05 11:38:00
3    2020-08-05 11:39:00
4    2020-08-05 11:40:00
5    2020-08-05 11:41:00
6    2020-08-05 11:43:00
7    2020-08-05 11:44:30
8    2020-08-05 11:46:00
9    2020-08-05 11:47:00
10   2020-08-05 11:48:00
11   2020-08-05 11:49:00
12   2020-08-05 11:50:00
13   2020-08-05 11:51:30
14   2020-08-05 11:53:30
15   2020-08-05 11:55:30
16   2020-08-05 11:57:00
17   2020-08-05 11:58:00
18   2020-08-05 12:00:00
19   2020-08-05 12:01:00
20   2020-08-05 12:03:00
21   2020-08-05 12:04:30
22   2020-08-05 12:06:00
23   2020-08-05 12:07:00
24   2020-08-05 12:09:30
25   2020-08-05 12:11:30
26   2020-08-05 12:13:00
27   2020-08-05 12:15:00
28   2020-08-05 12:16:30
29   2020-08-05 12:18:30
30   2020-08-05 12:20:00
31   2020-08-05 12:21:00
32   2020-08-05 12:22:30
33   2020-08-05 12:24:00
34   2020-08-05 12:25:30
35   2020-08-05 12:27:00
36   2020-08-05 12:28:30
37   2020-08-05 12:32:30
Name: stop_time, dtype: datetime64[ns]

In [115]:
# success! we only need one of these scheduled routes.
pd.concat([run1['stop_time'], run2['stop_time']], axis=1).diff()

,stop_time,stop_time
0,NaT,NaT
1,00:02:30,00:02:30
2,00:01:00,00:01:00
3,00:01:00,00:01:00
4,00:01:00,00:01:00
5,00:01:00,00:01:00
6,00:02:00,00:02:00
7,00:01:30,00:01:30
8,00:01:30,00:01:30
9,00:01:00,00:01:00
